In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM

from utils import load_Lie_weights
import sys
sys.path.insert(0,'..')

from core.Lie import LieLayer

Using TensorFlow backend.


Load weight of Lie transform:

In [3]:
state_dim = 12
W = load_Lie_weights('data/bath_bombs_005.txt', dim=state_dim)

('load weights: ', 'data/bath_bombs_005.txt')
('load matrix', 0)
start converting
(1,)
start reduction
end reduction
start kron
start kron
('load matrix', 1)
start converting
(12,)
start reduction
end reduction
start kron
start kron
('load matrix', 2)
start converting
(144,)
start reduction
end reduction
start kron
start kron
('load matrix', 3)
start converting
(1728,)
start reduction
end reduction
start kron
start kron
('load matrix', 4)


In [4]:
non_zero_sum = 0
total = 0
weights = []
for Wi in W:
    total+=Wi.shape[0]*Wi.shape[1]
    non_zero_sum += (Wi!=0).sum()
print('total weight count:',total)
print('non-zero weight count:',non_zero_sum)

plt.hist()

('total weight count:', 22620)
('non-zero weight count:', 199)


TypeError: hist() takes at least 1 argument (0 given)

Build a Lie transformed--based neural network

In [21]:
# user-defind loss function
def custom_mse(yTrue,yPred):
    weights = np.zeros(state_dim)
    weights[10] = 1
    return K.mean(weights*K.square(yTrue-yPred))


# Lie transform based neural network
lie_map = LieLayer(output_dim = state_dim, input_shape = (state_dim,), order = 3)
model = Sequential()
for i in range(200):
    model.add(lie_map)
opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.99,
                            beta_2=0.99999, epsilon=1e-1, decay=0.0)
model.compile(loss=custom_mse, optimizer=opt)

# load pre-fitted weights
lie_map.set_weights(W)

In [22]:
# bath bomb concentrations: data set
X0 = np.loadtxt('data/bath_bombs_concentrations.txt', delimiter=',')
print(X0.shape)

(10, 12)


Calculate stability of bath bomb product (ideally should be equal to ones):

In [23]:
X = model.predict(X0)
X[:, -2]

array([0.9240657 , 1.3650838 , 1.3204026 , 0.07104913, 1.0523348 ,
       1.2675126 , 1.409016  , 0.9126697 , 1.4060024 , 0.9584267 ],
      dtype=float32)

Remove SLES from components (modified input):

In [24]:
X0_bath_fizzies = X0.copy()
X0_bath_fizzies[:, [4, 7]] = 0 # remove SLES

X_bf = model.predict(X0_bath_fizzies)
(X_bf/(X+1e-10))[:, -2] # compare stability

array([0.9339177 , 0.9439575 , 0.9300925 , 0.9993514 , 0.9141857 ,
       0.9431441 , 0.9437397 , 0.96449965, 0.9438261 , 0.9477873 ],
      dtype=float32)

One have to change concentration of water to remain stability of the product at the same level:

In [1]:
water = np.array([0.64, 0.84, 0.88, 0.1 , 1.14, 0.76, 0.82, 0.36, 0.82, 0.5 ])
X0_bath_fizzies[:, 6] = water;
X0_bath_fizzies[:,-1] = water/10;

X_bf = model.predict(X0_bath_fizzies)
(X_bf/(X+1e-10))[:, -2]

NameError: name 'np' is not defined

To fit model a standart interface is available

In [8]:
X1 = X0.copy()
X1[:, [4,6,7]] = 0
X1[:, 10] = 1
model.fit(X0, X1, nb_epoch=3, verbose=1)

Epoch 1/10
10/10 [==============================] - 3s 329ms/step - loss: 0.0127
Epoch 2/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0530
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0081
Epoch 4/10
10/10 [==============================] - 0s 9ms/step - loss: 0.0135
Epoch 5/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0138
Epoch 6/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0077
Epoch 7/10
10/10 [==============================] - 0s 9ms/step - loss: 0.0045
Epoch 8/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0038
Epoch 9/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0041
Epoch 10/10
10/10 [==============================] - 0s 9ms/step - loss: 0.0047
